# Урок 2. Предварительная обработка текста

Перед тем, как обучать модель, данные следует специальным образом подготовить. Подготовка данных включает в себя несколько обязательных этапов:

* удалить все нерелевантные символы (например, любые символы, не относящиеся к цифро-буквенным).
* токенизировать текст, разделив его на индивидуальные слова (токены).
* удалить нерелевантные слова — например, упоминания в Twitter или URL-ы.
* перевести все символы в нижний регистр для того, чтобы слова «hello», «Hello» и «HELLO» были схлопнуты в один токен.
* исправление ошибок ("молоко" и "молако" - одно слово, но разные токены, не надо так).
* лемматизация - перевод слова в нормальную (словарную) форму (например, «машина» вместо «машиной»). Существительные должны быть приведены к единственному числу именительного падежа, глаголы - инфинитив и т.д.
* стемминг - процедура, когда от слова переходим к его корню ("помыть" и "мытый" - корень "мыт"). То есть все "помытые" заменяем на "мыт".

Все эти приёмы нужно применять с осторожностью и внимательно следить за тем, как тот или иной приём, применённый к исходному тексту, влияет на качество решения задачи (например, выявлению спама).

Для демонстрации всех этих приёмов загрузим корпус (набор текстов) с твитами о продуктах. Для каждого твита размечена эмоциональная окраска - позитивная, нейтральная или негативная. Примечание: для  обработки текста воспользуемся библиотекой nltk, которая [доступна в anaconda](https://anaconda.org/anaconda/nltk).

In [ ]:
import nltk
import string

# дополнительный словарь со знаками пунктуации
nltk.download('punkt', download_dir='.')

[nltk_data] Error loading punkt: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>


False

Загружаем исходные данные

In [ ]:
import pandas as pd

df = pd.read_csv('data/brand_tweets.csv', sep=',', encoding='utf8')
# удаляем строки, в которых отсутствует текст твита
df.drop(df[df.tweet_text.isnull()].index, inplace=True)
print(df.shape)

df.head()

(3904, 3)


,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


Для начала разделим текст на отдельные слова с помощью библиотечной функции `word_tokenize`, на примере первого документа в корпусе

In [ ]:
sample_str = df.tweet_text.values[0]

print('== Исходный текст== \n%s\n\n' % sample_str)

tokenized_str = nltk.word_tokenize(sample_str)
print('== Токенизированный текст==\n%s' % tokenized_str)

== Исходный текст== 
.@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead!  I need to upgrade. Plugin stations at #SXSW.


== Токенизированный текст==
['.', '@', 'wesley83', 'I', 'have', 'a', '3G', 'iPhone', '.', 'After', '3', 'hrs', 'tweeting', 'at', '#', 'RISE_Austin', ',', 'it', 'was', 'dead', '!', 'I', 'need', 'to', 'upgrade', '.', 'Plugin', 'stations', 'at', '#', 'SXSW', '.']


Отфильтруем знаки пунктуации, токены приведём к нижнему регистру

In [ ]:
tokens = [i.lower() for i in tokenized_str if ( i not in string.punctuation )]
print(tokens)

['wesley83', 'i', 'have', 'a', '3g', 'iphone', 'after', '3', 'hrs', 'tweeting', 'at', 'rise_austin', 'it', 'was', 'dead', 'i', 'need', 'to', 'upgrade', 'plugin', 'stations', 'at', 'sxsw']


Удаляем стоп-слова, список которых для русского языка можно получить как `stop_words = nltk.corpus.stopwords.words('russian')`. Стоп-слова это "мусорные" слова которые встречаются чрезычайно часто (в каждом предложении) поэтому не несут в себе никакой информации. Такие слова, вобщем-то, нужны только для красивой речи и поэтому можем их смело удалять из текста. Например, этот список стоп-слов я нагуглил в интернете.

In [ ]:
stop_words = [
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd",
    'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers',
    'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which',
    'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been',
    'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if',
    'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between',
    'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out',
    'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why',
    'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not',
    'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'shold',
    "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't",
    'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't",
    'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't",
    'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"
]

filtered_tokens = [i for i in tokens if ( i not in stop_words )]

print(filtered_tokens)

['wesley83', '3g', 'iphone', '3', 'hrs', 'tweeting', 'rise_austin', 'dead', 'need', 'upgrade', 'plugin', 'stations', 'sxsw']


Реализуем пайплайн в виде функции, при помощи которой обработаем все текстовые описания. Для каждого описания
* проводим токенизацию
* удаляем пунктуацию
* приводим к нижнему регистру
* удаляем стоп-слова

In [ ]:
def tokenize_text(raw_text: str):
    """Функция для токенизации текста
    
    :param raw_text: исходная текстовая строка
    """
    tokenized_str = nltk.word_tokenize(raw_text)
    tokens = [i.lower() for i in tokenized_str if ( i not in string.punctuation )]
    filtered_tokens = [i for i in tokens if ( i not in stop_words )]
    return filtered_tokens

# применяем функцию в датафрейму с помощью метода .apply()
tokenized_tweets= df.tweet_text.apply(tokenize_text)

# добавляем новую колонку в исходный датафрейм
df = df.assign(
    tokenized=tokenized_tweets
)

df.tokenized.head()

0    [wesley83, 3g, iphone, 3, hrs, tweeting, rise_...
1    [jessedee, know, fludapp, awesome, ipad/iphone...
2    [swonderlin, wait, ipad, 2, also, should, sale...
3    [sxsw, hope, year, 's, festival, n't, crashy, ...
4    [sxtxstate, great, stuff, fri, sxsw, marissa, ...
Name: tokenized, dtype: object

Следующий шаг - привести слово в нормальную (словарную) форму. Для русского языка мы уже проводили нормализацию (в занятии из цикла по Аналитике начального уровня) с помощью модуля pyMorphy, который отлично подходит для русского языка.

<pre>
import pymorphy2

morph = pymorphy2.MorphAnalyzer()
parsed_token = morph.parse(word)
normal_form = parsed_token[0].normal_form
</pre>

В силу того, что наши твиты на англиском языке, то этап нормализации не слишком актуален.

В результате токенизации мы преобразовали текст в последовательность токенов. Однако алгоритмы машинного обучения работают не с токенами, а с числами. Процесс перехода от токенов к числам наывается векторизацией, в следующем уроке мы поговорим об одном из самых простых приёмов векторизации - Bag of Words;